## Bilder Bauen mit dem Diagrams package
* Hilfe unter dem unten angemerkten link

## Den Diagramm Namen als erstes bei Änderungen anpassen um zu verhindern das bestehende Versionen überschritten werden

https://diagrams.mingrammer.com/docs/getting-started/installation

In [7]:
from diagrams import Cluster, Diagram, Edge
from diagrams.aws.compute import Lambda
from diagrams.aws.database import Dynamodb
from diagrams.aws.storage import S3
from diagrams.aws.integration import Eventbridge
from diagrams.aws.general import InternetGateway, InternetAlt1
from diagrams.aws.security import Cognito
from diagrams.generic.device import Mobile
from diagrams.programming.language import Python

with Diagram("BackendAPICalls_V1_0", show=False, direction="LR"):
    with Cluster('External API'):
        restAPI = InternetAlt1("REST API")
    
    EventSchedule = Eventbridge('Data Load Scheduler')
    
    with Cluster('Backend'):

        
        with Cluster("API Call Services"):
            Arbeitsagentur = Lambda("Arbeitsagentur Data Load") 
            apicall_group = [Arbeitsagentur]

        

        with Cluster('Data Storage'):
            DocumentDB = Dynamodb("Job Data DB")
            LearnBucket = S3("Learning Data")
            ModelBucket = S3('ML Model Storage')
        #ModelBucket = S3("ML Model")
        
        createManifest = Lambda("Create Sagemaker Manifest")
        mlops = Lambda('Perform Skill Extraction')

        with Cluster("Frontend API Functions"):
            frontendfunctions = [
                Lambda("GetSkills"),
                Lambda("GetJobs")
            ]
        frontendAPI = InternetGateway("Frontend API Gateway")
        authenticate = Cognito('App Authentification')
    with Cluster('Local'):
        pythonic = Python("ML Training")
            
    with Cluster('Frontend'):
        endgeraet = Mobile('Client')
    
    restAPI >> Edge(label="GET Request") >> apicall_group
    EventSchedule >> Edge(style='dotted') >> apicall_group
    EventSchedule >> Edge(style='dotted')>> createManifest
    DocumentDB >> Edge(style='invis') >> LearnBucket >> Edge(style='invis') >> ModelBucket
    apicall_group >> DocumentDB
    mlops - Edge(style='dashed', color='darkgreen') - ModelBucket
    mlops >> Edge(label='') << DocumentDB
    DocumentDB >> createManifest >> LearnBucket 
    DocumentDB >> frontendfunctions
    frontendfunctions >> Edge() << frontendAPI
    pythonic << LearnBucket
    pythonic >> ModelBucket
    frontendAPI << Edge() >> endgeraet
    authenticate << Edge() >> endgeraet
    frontendAPI << Edge() >> authenticate
    
    
    